torch.cuda.get_device_capability() 함수를 사용하여 현재 CUDA 장치의 major 버전과 minor 버전을 조회합니다.

In [2]:
# !pip uninstall torch


^C


In [3]:
# !pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://download.pytorch.org/whl/cu117
     ---------------------------------------- 0.0/2.3 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 GB 11.6 MB/s eta 0:03:23
     ---------------------------------------- 0.0/2.3 GB 10.5 MB/s eta 0:03:44
     ---------------------------------------- 0.0/2.3 GB 10.1 MB/s eta 0:03:53
     ---------------------------------------- 0.0/2.3 GB 10.0 MB/s eta 0:03:55
     ---------------------------------------- 0.0/2.3 GB 10.6 MB/s eta 0:03:40
     ---------------------------------------- 0.0/2.3 GB 10.4 MB/s eta 0:03:45
     ---------------------------------------- 0.0/2.3 GB 10.3 MB/s eta 0:03:47
     ---------------------------------------- 0.0/2.3 GB 10.2 MB/s eta 0:03:50
     ---------------------------------------- 0.0/2.3 GB 10.2 MB/s eta 0:03:50
     ---------------------------------------- 0.0/2.3 GB 10.3 MB/s eta 0:03:48
     ---------------------------------------- 0.0/2.3 GB 10.1 MB/s eta 0:03:52
 

  You can safely remove it manually.
  You can safely remove it manually.


In [5]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB 9.6 MB/s eta 0:04:34
     ---------------------------------------- 0.0/2.6 GB 9.5 MB/s eta 0:04:37
     ---------------------------------------- 0.0/2.6 GB 9.7 MB/s eta 0:04:32
     ---------------------------------------- 0.0/2.6 GB 9.6 MB/s eta 0:04:32
     ---------------------------------------- 0.0/2.6 GB 10.1 MB/s eta 0:04:18
     ---------------------------------------- 0.0/2.6 GB 10.1 MB/s eta 0:04:20
     ---------------------------------------- 0.0/2.6 GB 9.7 MB/s eta 0:04:31
     ---------------------------------------- 0.0/2.6 GB 9.7 MB/s eta 0:04:30
     ---------------------------------------- 0.0/2.6 GB 9.6 MB/s eta 0:04:32
     ---------------------------------------- 0.0/2.6 GB 9.7 MB/s eta 0:04:31
     ---------------------------------------- 0.0/2.6 GB 9.7 MB/s eta 0:04:31
     -----

In [1]:
import torch

# CUDA 가 사용 가능한지 확인
if torch.cuda.is_available():
    print("CUDA is available.")
    major_version, minor_version = torch.cuda.get_device_capability()
    print(f"CUDA Capability: {major_version}.{minor_version}")
else:
    print("CUDA is not available.")

CUDA is available.
CUDA Capability: 7.5


In [2]:
import torch

# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(7, 5)

unsloth 라이브러리와 관련 디펜던시를 설치하는 과정을 설명합니다.

- Colab 환경에서 torch 버전 2.2.1과 호환되지 않는 패키지를 회피하기 위해 unsloth 라이브러리를 별도로 설치합니다.
- GPU의 종류(신형 또는 구형)에 따라 조건부로 필요한 패키지들을 설치합니다.
    - 신형 GPU(Ampere, Hopper 등)의 경우, packaging, ninja, einops, flash-attn, xformers, trl, peft, accelerate, bitsandbytes 패키지를 의존성 없이 설치합니다.
    - 구형 GPU(V100, Tesla T4, RTX 20xx 등)의 경우, xformers, trl, peft, accelerate, bitsandbytes 패키지를 의존성 없이 설치합니다.
- 설치 과정에서 발생하는 출력을 숨기기 위해 %%capture 매직 커맨드를 사용합니다.

In [3]:
%%capture
# Colab에서 torch 2.2.1을 사용하고 있으므로, 패키지 충돌을 방지하기 위해 별도로 설치해야 합니다.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용하세요.
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

### Unsloth

- Unsloth는 Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes 등을 지원합니다. 그리고 Yi, Qwen(llamafied), Deepseek, 모든 Llama, Mistral 파생 아키텍처도 지원합니다.

- Unsloth는 16비트 LoRA 또는 4비트 QLoRA를 지원합니다. 둘 다 2배 빠릅니다.

- max_seq_length는 kaiokendev의 방법을 통해 자동으로 RoPE 스케일링을 하기 때문에 어떤 값으로도 설정할 수 있습니다.

FastLanguageModel.from_pretrained 함수를 사용하여 사전 훈련된 언어 모델을 로드하는 과정을 설명합니다.

- 최대 시퀀스 길이(max_seq_length)를 설정하여 모델이 처리할 수 있는 입력 데이터의 길이를 지정합니다.
- 데이터 타입(dtype)은 자동 감지되거나, 특정 하드웨어에 최적화된 형식(Float16, Bfloat16)으로 설정할 수 있습니다.
- 4비트 양자화(load_in_4bit) 옵션을 사용하여 메모리 사용량을 줄일 수 있으며, 이는 선택적입니다.
- 사전 정의된 4비트 양자화 모델 목록(fourbit_models)에서 선택하여 다운로드 시간을 단축하고 메모리 부족 문제를 방지할 수 있습니다.
- FastLanguageModel.from_pretrained 함수를 통해 모델과 토크나이저를 로드하며, 이때 모델 이름(model_name), 최대 시퀀스 길이, 데이터 타입, 4비트 로딩 여부를 매개변수로 전달합니다.
- 선택적으로, 특정 게이트 모델을 사용할 경우 토큰(token)을 제공할 수 있습니다.

In [5]:
!pip install triton


ERROR: Could not find a version that satisfies the requirement triton (from versions: none)
ERROR: No matching distribution found for triton


In [6]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenㄴizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    model_name="beomi/Llama-3-Open-Ko-8B-Instruct-preview",  # 모델 이름을 설정합니다.
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

ModuleNotFoundError: No module named 'triton'